image.png

In [1]:
import tensorflow as tf
import numpy as np # 추가
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report # 추가


2025-11-24 06:28:53.615454: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-24 06:28:57.123189: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-24 06:29:22.020933: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 사용자 설정
train_dir = '/workspace/merge_data_binary/new_data/img_binary'
val_dir = '/workspace/merge_data_binary/new_data_val/img_binary'



target_size = (224, 224)
batch_size = 32
seed = 42

positive_class = 'happy'  # 'happy' -> 1, 나머지 폴더 -> 0

# helper: 디렉터리를 스캔해서 파일경로와 라벨(1:happy, 0:others) DataFrame 생성
def make_dataframe_from_dir(base_dir, positive_class='happy'):
    rows = []
    for class_name in sorted(os.listdir(base_dir)):
        class_path = os.path.join(base_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        label = 1 if class_name == positive_class else 0
        for fname in sorted(os.listdir(class_path)):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                rows.append({
                    'filepath': os.path.join(class_path, fname),
                    'label': label,
                    'orig_class': class_name
                })
    df = pd.DataFrame(rows)
    return df

# DataFrame 생성
train_df = make_dataframe_from_dir(train_dir, positive_class=positive_class)
val_df = make_dataframe_from_dir(val_dir, positive_class=positive_class)

print("Train samples:", len(train_df), "Val samples:", len(val_df))
print("Train class distribution:\n", train_df['label'].value_counts())
print("Val class distribution:\n", val_df['label'].value_counts())

# ImageDataGenerator (전처리/증강)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# flow_from_dataframe 생성 (class_mode='raw'로 0/1 레이블 반환)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    shuffle=True,
    seed=seed
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False,  # 평가 시 순서 유지
    seed=seed
)


Train samples: 9550 Val samples: 1151
Train class distribution:
 label
1    5202
0    4348
Name: count, dtype: int64
Val class distribution:
 label
0    852
1    299
Name: count, dtype: int64
Found 9550 validated image filenames.
Found 1151 validated image filenames.


In [3]:

# 모델 정의 (원본 코드와 동일)
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                           input_shape=(target_size[0], target_size[1], 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()

# 학습
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
)



I0000 00:00:1763965828.712182  609730 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42799 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:53:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

...

 83/299 ━━━━━━━━━━━━━━━━━━━━ 9:01 3s/step - accuracy: 0.5591 - auc: 0.5768 - loss: 0.6979

2025-11-24 06:34:41.454074: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-24 06:34:41.579451: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-24 06:34:42.062591: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-24 06:34:42.191703: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-24 06:34:42.320517: E external/local_xla/xla/stream_

299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5919 - auc: 0.6223 - loss: 0.6722

2025-11-24 06:45:44.281709: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-24 06:45:44.408944: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-24 06:45:44.808704: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-24 06:45:44.940832: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-24 06:45:45.069704: E external/local_xla/xla/stream_

299/299 ━━━━━━━━━━━━━━━━━━━━ 910s 3s/step - accuracy: 0.6190 - auc: 0.6591 - loss: 0.6507 - val_accuracy: 0.5065 - val_auc: 0.6862 - val_loss: 0.7576
Epoch 2/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 737s 2s/step - accuracy: 0.6595 - auc: 0.7143 - loss: 0.6136 - val_accuracy: 0.4440 - val_auc: 0.6877 - val_loss: 0.8803
Epoch 3/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1088s 4s/step - accuracy: 0.6817 - auc: 0.7441 - loss: 0.5916 - val_accuracy: 0.6064 - val_auc: 0.7186 - val_loss: 0.6685
Epoch 4/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 735s 2s/step - accuracy: 0.6918 - auc: 0.7553 - loss: 0.5816 - val_accuracy: 0.6325 - val_auc: 0.7186 - val_loss: 0.6460
Epoch 5/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1112s 4s/step - accuracy: 0.7088 - auc: 0.7761 - loss: 0.5630 - val_accuracy: 0.5752 - val_auc: 0.7142 - val_loss: 0.7409
Epoch 6/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 668s 2s/step - accuracy: 0.7133 - auc: 0.7836 - loss: 0.5543 - val_accuracy: 0.5621 - val_auc: 0.7154 - val_loss: 0.7675
Epoch 7/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 164

In [5]:
# --- 검증 데이터셋에 대한 성능 평가 지표 추가 ---
print("--- 검증 데이터셋 성능 평가 시작 ---")

# 1. 검증 데이터셋의 실제 레이블 (True Labels) 추출
if hasattr(validation_generator, 'labels'):
    validation_labels = validation_generator.labels
elif hasattr(validation_generator, 'classes'):
    validation_labels = validation_generator.classes
else:
    validation_labels = val_df['label'].values

# 2. 모델 예측 (Predictions) 수행
validation_generator.reset()
pred_probabilities = model.predict(validation_generator, steps=validation_steps, verbose=1)

# 3. 예측 확률을 이진 클래스 (0 또는 1)로 변환
pred_classes = (pred_probabilities > 0.5).astype(int).flatten()

# 4. classification_report 출력
class_names = ['others', 'happy']
report = classification_report(validation_labels, pred_classes, target_names=class_names, digits=4)
print("\n--- Classification Report (Validation Set) ---\n")
print(report)
print("--- 검증 데이터셋 성능 평가 완료 ---")



--- 검증 데이터셋 성능 평가 시작 ---


36/36 ━━━━━━━━━━━━━━━━━━━━ 126s 4s/step

--- Classification Report (Validation Set) ---

              precision    recall  f1-score   support

      others     0.8712    0.5317    0.6603       852
       happy     0.3677    0.7759    0.4989       299

    accuracy                         0.5951      1151
   macro avg     0.6194    0.6538    0.5796      1151
weighted avg     0.7404    0.5951    0.6184      1151

--- 검증 데이터셋 성능 평가 완료 ---
